# Data-informed parameter synthesis for population Markov chains, notebook1

This notebook provides two first parts of analysis presented in the paper. 
1. [Creation of the models](#one)
2. [Parameter synthesis using prism](#two)

The second two parts are in following notebook.

Before running, please add [prism](prismmodelchecker.org) to PATH and check if it runs correctly.

There are 4 things to setup in the notebook:
1. `prism_path` -- path of prism - in order to import and call the prism properly please set up the paths (e.g. calling the prism outside )
2. `agents_quantities` -- set of agents number to be considered - set numbers for which you want to run analysis

## IMPORTS, PATHS, and SETTINGS

if a package fails to load due to it is not installed please uncomment and use following code with respective package name

In [1]:
#import sys
#!{sys.executable} -m pip install <package_name>

In [2]:
import struct
print("You are running "+ str(struct.calcsize("P") * 8)+"bit Python, please verify that installed z3 is compatible")

You are running 64bit Python, please verify that installed z3 is compatible


In [3]:
#imports evaluation
import time

In [4]:
#imports for creation of models
import math
import sys

In [5]:
#imports for running prism
import glob, os
import subprocess
import re

In [6]:
# import enhaced print
import socket

In [7]:
cwd = os.getcwd()

In [8]:
os.getcwd()

'D:\\Git\\hsb2019\\code'

Set path for prism

In [9]:
prism_path = ("c:/Program Files/prism-4.4/bin") #windows

#prism_path = ("/home/matej/prism-4.4-linux64/bin") #Freya

#prism_path = '/Users/tatjanapetrov/MYprism-4.4-osx64/bin' # mac@Uni
#os.environ['PATH'] = os.environ['PATH']+str(prism_path)

Checking if the prism in the PATH

In [10]:
if "prism" not in os.environ["PATH"]:
    print("prism was probably not in PATH, adding it there")
    if "wind" in platform.system().lower():
        os.environ["PATH"] = os.environ["PATH"]+";"+prism_path
    else:
        os.environ["PATH"] = os.environ["PATH"]+":"+prism_path

Set agents quantities to be examined

In [11]:
agents_quantities = [2,3,5,10]

<a name="one"></a>

## 1. CREATION OF THE MODELS

In this section are created two-param (p,q) and multiparametric models. Detailed description can be found in the paper in Section 2. For all models created in the notebook the bisimulation quotient has been applied. So to speeak, the states (1,0) and (0,1) are equal since the agent are the considered agents are identical. Hence we have merged those states.

The difference between synchronous, semisynchronous, and asynchronous models is in two parts of the model:
1. first attempt of the agent
2. second attempt 

synchronous models use synchronous semantics in the first attempt so does in the second attempt, these models are purely synchronous and differ from the other models in multiparametric case. 

asynchronous models use asynchronous semantics in the first attempt so does in the second attempt, these models are NOT purely synchronous as they assume synchronisation of the agents after first attempt! These models are largest and provide multi-affine labelling, hence are usable for STORM model checker. 

models called semisynchronous use synchronous semantics in the first attempt and asynchronous semantics in the second attempt. These models use synchronous semantics where the agents are independent and provide more precise asynchronous semantic when the order of the action is needed - Example: multiparametric case, two agents attempt second try while 3 agents has already succeeded - the first agent has probability q3 to succeeded, and the second q4 only if the first succeeded. We need more than 1 transition to describe this phenomena.

In [12]:
def nCr(n,k):
    """ Return conbinatorial number n take k
    
    Parameters
    ----------
    n : int
    k : int
    """
    f = math.factorial
    return f(n) / f(k) / f(n-k)

### Creating models of synchronous two-param models

In [127]:
def create_synchronous_model(file,N):
    """ Creates synchronous model of *N* agents to a *file* with probabilities p and q in [0,1]. For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)
    
    first_attempt = []
    coeficient = ""

    for i in range(N+1):
        for j in range(N-i):
            coeficient = coeficient + "*p"
        for j in range(N-i,N):
            coeficient = coeficient + "*(1-p)"
        coeficient = str(nCr(N,i))+ coeficient
        first_attempt.append(coeficient)
        coeficient = ""
    #print(first_attempt)

    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")
    file.write("const double q;\n")
    file.write("\n" )

    # module here 
    file.write("module bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transition\n")
    file.write("       []   a0 = -1")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = -1 ")
    file.write("-> ")
    for i in range(N+1):
        file.write( first_attempt[i]+": " )
        for j in range(N):
            file.write("(a"+str(j)+"'="+ str( 1 if N-i>j else 2 )+")")
            if j<N-1:
                file.write(" & ")    
        if i<N:
            file.write(" + " )
    file.write(";\n")
    file.write("\n")
    
    # non-initial transition
    
    # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for ones in range(1,N):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if ones>j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    
        
        twos=N-ones
        #file.write("twos: {}".format(twos))
        
        for sucessses in range(0,twos+1):
            file.write(str(nCr(twos,sucessses)))
            for ok in range(sucessses):
                file.write("*q")
            for nok in range(twos-sucessses):
                file.write("*(1-q)")
            file.write(": ")
            
            for k in range(1,N+1):
                if k<=ones+sucessses:
                    if k==N:
                        file.write("(a"+str(k-1)+"'=1)")
                    else:
                        file.write("(a"+str(k-1)+"'=1) & ")
                elif k==N:
                    file.write("(a"+str(k-1)+"'=0)")
                else:
                    file.write("(a"+str(k-1)+"'=0) & ")
            if sucessses<twos:
                file.write(" + " ) 

        file.write(";\n")
    file.write("\n")

    # all twos transitions
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")

    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [128]:
for N in agents_quantities:
    create_synchronous_model("synchronous_parallel_"+str(N),N)

synchronous_parallel_2
synchronous_parallel_3
synchronous_parallel_5
synchronous_parallel_10


### Creating models of semisynchronous two-param models

In [13]:
def create_semisynchronous_model(file,N):
    """ Creates semisynchronous model of *N* agents to a *file* with probabilities p and q in [0,1]. For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)
    
    first_attempt = []
    coeficient = ""

    for i in range(N+1):
        for j in range(N-i):
            coeficient = coeficient + "*p"
        for j in range(N-i,N):
            coeficient = coeficient + "*(1-p)"
        coeficient = str(nCr(N,i))+ coeficient
        first_attempt.append(coeficient)
        coeficient = ""
    #print(first_attempt)

    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")
    file.write("const double q;\n")
    file.write("\n" )

    # module here 
    file.write("module bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transition\n")
    file.write("       []   a0 = -1")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = -1 ")
    file.write("-> ")
    for i in range(N+1):
        file.write( first_attempt[i]+": " )
        for j in range(N):
            file.write("(a"+str(j)+"'="+ str( 1 if N-i>j else 2 )+")")
            if j<N-1:
                file.write(" & ")    
        if i<N:
            file.write(" + " )
    file.write(";\n")
    file.write("\n")

    # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for i in range(N-1):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    

        file.write("q:")
        for j in range(N):
            file.write("(a"+str(j)+"'= "+str( 1 if i+1>=j else 2 ) +")"+str( " & " if j<N-1 else ""))
        file.write(" + ")
        file.write("1-q:")    
        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>=j else 2 ) +") & ")
        file.write("(a"+str(N-1)+"'= 0)")
        #print("N",N,"i",i)
        file.write(";\n")
    file.write("\n")


    # some ones, some twos transitions, some zeros transitions
    file.write("       // some ones, some twos, some zeros transitions\n")
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            file.write("       []   a0 = 1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 2")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 0")

            file.write(" -> ")    
            file.write("q: (a0' = 1)")
            for j in range(1,o+1):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+1,o+t):
                file.write(" & (a"+str(j)+"'= 2)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"'= 0)")

            file.write(" + ")
            file.write("1-q: (a0' = 1)")
            for j in range(1,o):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o,o+t-1):
                file.write(" & (a"+str(j)+"'= 2)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"'= 0)")
            file.write(";\n")
            
            # print("ones: "+str(o)," twos: "+str(t)," zeros: "+str(z))
    file.write("\n")

    # all twos transitions
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")

    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [14]:
for N in agents_quantities:
    create_semisynchronous_model("semisynchronous_parallel_"+str(N),N)

semisynchronous_parallel_2
semisynchronous_parallel_3
semisynchronous_parallel_5
semisynchronous_parallel_10


### Creating models of asynchronous two-param models

In [21]:
def create_asynchronous_model(file,N):
    """ Creates aynchronous model of *N* agents to a *file* with probabilities p and q in [0,1]. For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    
    
    Model meaning
    ----------
    params:
    N - number of agents (agents quantity)
    p - proprability to succeed in the first attempt
    q - proprability to succeed when getting help
    ai - state of agent i:  -1:init, 0:total_failure, 1:succes, 2:failure_after_first_attempt
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)
    
    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")
    file.write("const double q;\n")
    file.write("\n" )

    # module here 
    file.write("module bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transitions\n")
    
    
    # some -1, some 1
    file.write("       // some -1, some 1 transitions\n")
    for i in reversed(range(0,N)):
        #for k in range(1,N):
        file.write("       []   a0 = -1")
        for j in range(1,N):
            if j>i:
                file.write(" & a"+str(j)+" = 1 ")
            else:
                file.write(" & a"+str(j)+" = -1 ")
        file.write("-> ")

        file.write( "p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")

        for j in range(1,i):
            file.write(" & (a"+str(j)+"' = -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 1)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 1)")

        file.write( " + 1-p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")
            
        for j in range(1,i):
            file.write(" & (a"+str(j)+"'= -1)")
        for j in range( max(i,1),N-1):
            file.write(" & (a"+str(j)+"' = 1)")
        file.write(" & (a"+str(N-1)+"' = 2)")
        
        file.write(";\n")
        # file.write("i="+str(i)+" j="+str(j)+" \n")
    file.write("\n")
    
    # some -1, some 2
    file.write("       // some -1, some 2 transitions\n")
    for i in reversed(range(0,N-1)):
        #for k in range(1,N):
        file.write("       []   a0 = -1")
        for j in range(1,N):
            if j>i:
                file.write(" & a"+str(j)+" = 2")
            else:
                file.write(" & a"+str(j)+" = -1")
        file.write("-> ")

        file.write( "p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")

        for j in range(1,i):
            file.write(" & (a"+str(j)+"' = -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 1)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 2)")

        file.write( " + 1-p: " )
        if i==0:
            file.write("(a0' = 2)")
        else:
            file.write("(a0' = -1)")        
        for j in range(1,i):
            file.write(" & (a"+str(j)+"'= -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 2)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 2)")

        file.write(";\n")
        
    file.write("\n")
    
    # some -1, some 1, some 2
    file.write("       // some -1, some 1, some 2 transitions\n")
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            file.write("       []   a0 = -1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = -1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 2")

            file.write(" -> ")    
            if o>1:
                file.write("p: (a0' = -1)")
            else:
                file.write("p: (a0' = 1)")
            for j in range(1,o-1):
                file.write(" & (a"+str(j)+"'= -1)")
            for j in range(max(1,o-1),o+t):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"'= 2)")

            file.write(" + ")
            if o>1:
                file.write("1-p: (a0' = -1)")
            else:
                file.write("1-p: (a0' = 1)")
            for j in range(1,o-1):
                file.write(" & (a"+str(j)+"'= -1)")
            for j in range(max(1,o-1),o+t-1):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"'= 2)")
            file.write(";\n")
    file.write("\n")
    
    # not initial transition
    file.write("       //  not initial transitions\n")

    # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for i in range(N-1):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    

        file.write("q:")
        for j in range(N):
            file.write("(a"+str(j)+"'= "+str( 1 if i+1>=j else 2 ) +")"+str( " & " if j<N-1 else ""))
        file.write(" + ")
        file.write("1-q:")    
        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>=j else 2 ) +") & ")
        file.write("(a"+str(N-1)+"'= 0)")
        #print("N",N,"i",i)
        file.write(";\n")
    file.write("\n")


    # some ones, some twos transitions, some zeros transitions
    file.write("       // some ones, some twos, some zeros transitions\n")
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            file.write("       []   a0 = 1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 2")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 0")

            file.write(" -> ")    
            file.write("q: (a0' = 1)")
            for j in range(1,o+1):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+1,o+t):
                file.write(" & (a"+str(j)+"'= 2)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"'= 0)")

            file.write(" + ")
            file.write("1-q: (a0' = 1)")
            for j in range(1,o):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o,o+t-1):
                file.write(" & (a"+str(j)+"'= 2)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"'= 0)")
            file.write(";\n")
            
            # print("ones: "+str(o)," twos: "+str(t)," zeros: "+str(z))
    file.write("\n")

    # all twos transitions
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")

    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [22]:
for N in agents_quantities:
    create_asynchronous_model("asynchronous_"+str(N),N)

asynchronous_2
asynchronous_3
asynchronous_5
asynchronous_10


### Creating models of synchronous multiparam models

In [133]:
def create_multiparam_synchronous_model(file,N):
    """ Creates synchronous model of *N* agents to a *file* with probabilities p and q in [0,1]. For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)
    
    first_attempt = []
    coeficient = ""

    for i in range(N+1):
        for j in range(N-i):
            coeficient = coeficient + "*p"
        for j in range(N-i,N):
            coeficient = coeficient + "*(1-p)"
        coeficient = str(nCr(N,i))+ coeficient
        first_attempt.append(coeficient)
        coeficient = ""
    #print(first_attempt)

    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")

    for i in range(1,N):
        file.write("const double q"+str(i)+";\n")
    file.write("\n" )
    
    # module here 
    file.write("module bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transition\n")
    file.write("       []   a0 = -1")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = -1 ")
    file.write("-> ")
    for i in range(N+1):
        file.write( first_attempt[i]+": " )
        for j in range(N):
            file.write("(a"+str(j)+"'="+ str( 1 if N-i>j else 2 )+")")
            if j<N-1:
                file.write(" & ")    
        if i<N:
            file.write(" + " )
    file.write(";\n")
    file.write("\n")
    
    # non-initial transition
    
     # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for ones in range(1,N):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if ones>j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    
        
        twos=N-ones
        #file.write("twos: {}".format(twos))
        for sucessses in range(0,twos+1):
            file.write(str(nCr(twos,sucessses)))
            for ok in range(sucessses):
                file.write("*q{}".format(ones))
            for nok in range(twos-sucessses):
                file.write("*(1-q{})".format(ones))
            file.write(": ")
            
            for k in range(1,N+1):
                if k<=ones+sucessses:
                    if k==N:
                        file.write("(a"+str(k-1)+"'=1)")
                    else:
                        file.write("(a"+str(k-1)+"'=1) & ")
                elif k==N:
                    file.write("(a"+str(k-1)+"'=0)")
                else:
                    file.write("(a"+str(k-1)+"'=0) & ")
            if sucessses<twos:
                file.write(" + " ) 

        file.write(";\n")
    file.write("\n")

    # all twos transitions
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")

    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [134]:
for N in agents_quantities:
    create_multiparam_synchronous_model("multiparam_synchronous_parallel_"+str(N),N)

multiparam_synchronous_parallel_2
multiparam_synchronous_parallel_3
multiparam_synchronous_parallel_5
multiparam_synchronous_parallel_10


### Creating models of semisynchronous multi-param models

In [139]:
def create_multiparam_semisynchronous_model(file,N):
    """ Creates semisynchronous model of *N* agents to a *file* with probabilities p and multiple q-s in [0,1].
    For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)

    first_attempt = []
    coeficient = ""

    for i in range(N+1):
        for j in range(N-i):
            coeficient = coeficient + "*p"
        for j in range(N-i,N):
            coeficient = coeficient + "*(1-p)"
        coeficient = str(nCr(N,i))+ coeficient
        first_attempt.append(coeficient)
        coeficient = ""
    #print(first_attempt)

    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")


    for i in range(1,N):
        file.write("const double q"+str(i)+";\n")
    file.write("\n" )

    # module here 
    file.write("module multiparam_bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transition\n")
    file.write("       []   a0 = -1")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = -1 ")
    file.write("-> ")
    for i in range(N+1):
        file.write( first_attempt[i]+": " )
        for j in range(N):
            file.write("(a"+str(j)+"'="+ str( 1 if N-i>j else 2 )+")")
            if j<N-1:
                file.write(" & ")    
        if i<N:
            file.write(" + " )
    file.write(";\n")
    file.write("\n")

    # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for i in range(N-1):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    

        file.write("q"+str(i+1)+":")
        for j in range(N):
            file.write("(a"+str(j)+"'= "+str( 1 if i+1>=j else 2 ) +")"+str( " & " if j<N-1 else ""))
        file.write(" + ")
        file.write("1-q"+str(i+1)+":")    
        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>=j else 2 ) +") & ")
        file.write("(a"+str(N-1)+"'= 0)")
        #print("N",N,"i",i)
        file.write(";\n")
    file.write("\n")


    # some ones, some twos transitions, some zeros transitions
    file.write("       // some ones, some twos, some zeros transitions\n")
    i=0
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            i=i+1
            file.write("       []   a0 = 1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 2")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 0")

            file.write(" -> ")    
            file.write("q"+str(o)+": (a0' = 1)")
            for j in range(1,o+1):
                file.write(" & (a"+str(j)+"' = 1)")
            for j in range(o+1,o+t):
                file.write(" & (a"+str(j)+"' = 2)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"' = 0)")

            file.write(" + ")
            file.write("1-q"+str(o)+": (a0' = 1)")
            for j in range(1,o):
                file.write(" & (a"+str(j)+"' = 1)")
            for j in range(o,o+t-1):
                file.write(" & (a"+str(j)+"' = 2)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"' = 0)")
            file.write(";\n")

            # print("ones: "+str(o)," twos: "+str(t)," zeros: "+str(z))
    file.write("\n")


    #all twos transition
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")


    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [126]:
for N in agents_quantities:
    create_multiparam_semisynchronous_model("multiparam_semisynchronous_parallel_"+str(N),N)

multiparam_semisynchronous_parallel_2
multiparam_semisynchronous_parallel_3
multiparam_semisynchronous_parallel_5
multiparam_semisynchronous_parallel_10


### Creating models of asynchronous multi-param models

In [23]:
def create_multiparam_asynchronous_model(file,N):
    """ Creates semisynchronous model of *N* agents to a *file* with probabilities p and multiple q-s in [0,1].
    For more information see paper.
    
    Parameters
    ----------
    file : string - filename with extesion
    N : int - agent quantity
    """
    filename = file.split(".")[0] 
    file = open(filename+".pm","w") 
    print(filename)
    
    first_attempt = []
    coeficient = ""

    for i in range(N+1):
        for j in range(N-i):
            coeficient = coeficient + "*p"
        for j in range(N-i,N):
            coeficient = coeficient + "*(1-p)"
        coeficient = str(nCr(N,i))+ coeficient
        first_attempt.append(coeficient)
        coeficient = ""
    #print(first_attempt)

    # start here 
    file.write("dtmc \n \n") 
    file.write("const double p;\n")


    for i in range(1,N):
        file.write("const double q"+str(i)+";\n")
    file.write("\n" )

    # module here 
    file.write("module multiparam_bees_"+str(N)+"\n" )
    file.write("       // ai - state of agent i:  -1:init 0:total_failure 1:succes 2:failure_after_first_attempt\n")
    for i in range(N):
        file.write("       a"+str(i)+" : [-1..2] init -1; \n")
    file.write("       b : [0..1] init 0; \n")
    file.write("\n" )

    # transitions here 
    # initial transition
    file.write("       //  initial transitions\n")
    
    
    # some -1, some 1
    file.write("       // some -1, some 1 transitions\n")
    for i in reversed(range(0,N)):
        #for k in range(1,N):
        file.write("       []   a0 = -1")
        for j in range(1,N):
            if j>i:
                file.write(" & a"+str(j)+" = 1 ")
            else:
                file.write(" & a"+str(j)+" = -1 ")
        file.write("-> ")

        file.write( "p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")

        for j in range(1,i):
            file.write(" & (a"+str(j)+"' = -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 1)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 1)")

        file.write( " + 1-p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")
            
        for j in range(1,i):
            file.write(" & (a"+str(j)+"'= -1)")
        for j in range( max(i,1),N-1):
            file.write(" & (a"+str(j)+"' = 1)")
        file.write(" & (a"+str(N-1)+"' = 2)")
        
        file.write(";\n")
        # file.write("i="+str(i)+" j="+str(j)+" \n")
    file.write("\n")
    
    # some -1, some 2
    file.write("       // some -1, some 2 transitions\n")
    for i in reversed(range(0,N-1)):
        #for k in range(1,N):
        file.write("       []   a0 = -1")
        for j in range(1,N):
            if j>i:
                file.write(" & a"+str(j)+" = 2")
            else:
                file.write(" & a"+str(j)+" = -1")
        file.write("-> ")

        file.write( "p: " )
        if i==0:
            file.write("(a0' = 1)")
        else:
            file.write("(a0' = -1)")

        for j in range(1,i):
            file.write(" & (a"+str(j)+"' = -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 1)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 2)")

        file.write( " + 1-p: " )
        if i==0:
            file.write("(a0' = 2)")
        else:
            file.write("(a0' = -1)")        
        for j in range(1,i):
            file.write(" & (a"+str(j)+"'= -1)")
        if i>0:
            file.write(" & (a"+str(i)+"' = 2)")       
        for j in range(i+1,N):
            file.write(" & (a"+str(j)+"' = 2)")

        file.write(";\n")
        
    file.write("\n")
    
    # some -1, some 1, some 2
    file.write("       // some -1, some 1, some 2 transitions\n")
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            file.write("       []   a0 = -1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = -1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 2")

            file.write(" -> ")    
            if o>1:
                file.write("p: (a0' = -1)")
            else:
                file.write("p: (a0' = 1)")
            for j in range(1,o-1):
                file.write(" & (a"+str(j)+"'= -1)")
            for j in range(max(1,o-1),o+t):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"'= 2)")

            file.write(" + ")
            if o>1:
                file.write("1-p: (a0' = -1)")
            else:
                file.write("1-p: (a0' = 1)")
            for j in range(1,o-1):
                file.write(" & (a"+str(j)+"'= -1)")
            for j in range(max(1,o-1),o+t-1):
                file.write(" & (a"+str(j)+"'= 1)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"'= 2)")
            file.write(";\n")
    file.write("\n")

    # some ones, some zeros transitions
    file.write("       // some ones, some zeros transitions\n")
    for i in range(N+1):
        file.write("       []   a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(" -> ")    

        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>j else 0 ) +") & ")
        # print("N",N,"i",i)
        file.write("(a"+str(N-1)+"'= "+str( 1 if i == N else 0 )+")")
        file.write(" & (b'=1);\n")
    file.write("\n")


    # some ones, some twos transitions
    file.write("       // some ones, some twos transitions\n")
    for i in range(N-1):
        file.write("       []   a0 = 1")
        for j in range(1,N):
            #print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
            #print(" & a"+str(j)+" = "+str( 1 if i>=j else 2 ))
        file.write(" -> ")    

        file.write("q"+str(i+1)+":")
        for j in range(N):
            file.write("(a"+str(j)+"'= "+str( 1 if i+1>=j else 2 ) +")"+str( " & " if j<N-1 else ""))
        file.write(" + ")
        file.write("1-q"+str(i+1)+":")    
        for j in range(N-1):
            file.write("(a"+str(j)+"'= "+str( 1 if i>=j else 2 ) +") & ")
        file.write("(a"+str(N-1)+"'= 0)")
        #print("N",N,"i",i)
        file.write(";\n")
    file.write("\n")


    # some ones, some twos transitions, some zeros transitions
    file.write("       // some ones, some twos, some zeros transitions\n")
    i=0
    for o in range(1,N-1):
        #file.write("help")
        for t in range(1,N-o):
            z=N-t-o
            i=i+1
            file.write("       []   a0 = 1")
            for j in range(1,o):
                file.write(" & a"+str(j)+" = 1")
            for j in range(o,o+t):
                file.write(" & a"+str(j)+" = 2")
            for j in range(o+t,o+t+z):
                file.write(" & a"+str(j)+" = 0")

            file.write(" -> ")    
            file.write("q"+str(o)+": (a0' = 1)")
            for j in range(1,o+1):
                file.write(" & (a"+str(j)+"' = 1)")
            for j in range(o+1,o+t):
                file.write(" & (a"+str(j)+"' = 2)")
            for j in range(o+t,o+t+z):
                file.write(" & (a"+str(j)+"' = 0)")

            file.write(" + ")
            file.write("1-q"+str(o)+": (a0' = 1)")
            for j in range(1,o):
                file.write(" & (a"+str(j)+"' = 1)")
            for j in range(o,o+t-1):
                file.write(" & (a"+str(j)+"' = 2)")
            for j in range(o+t-1,o+t+z):
                file.write(" & (a"+str(j)+"' = 0)")
            file.write(";\n")

            # print("ones: "+str(o)," twos: "+str(t)," zeros: "+str(z))
    file.write("\n")


    #all twos transition
    file.write("       // all twos transition\n")
    file.write("       []   a0 = 2")
    for i in range(1,N):
        file.write(" & a"+str(i)+" = 2 ")
    file.write("-> ")
    for i in range(N-1):
        file.write("(a"+str(i)+"'= 0) & ")
    file.write("(a"+str(N-1)+"'= 0)")
    file.write(";\n")
    file.write("endmodule \n") 

    file.write("\n")


    # rewards here 
    file.write('rewards "coin_flips" \n')
    for i in range(N+1):
        file.write("       a0 = "+str( 0 if i==0 else 1 ))
        for j in range(1,N):
            # print("N",N,"i",i,"j",j)
            file.write(" & a"+str(j)+" = "+str( 1 if i>j else 0 ))
        file.write(":"+str(i)+";\n")
    file.write("endrewards \n")
    file.close()

In [137]:
for N in agents_quantities:
    create_multiparam_asynchronous_model("multiparam_asynchronous_"+str(N),N)

multiparam_asynchronous_2
multiparam_asynchronous_3
multiparam_asynchronous_5
multiparam_asynchronous_10


### Creating properties files

In [25]:
def create_properties(N):
    """ Creates property file of reaching each BSCC of the model of *N* agents as prop_<N>.pctl file.
    For more information see paper.
    
    Parameters
    ----------
    N : int - agent quantity
    """
    
    file = open("prop_"+str(N)+".pctl","w") 
    print("prop_{}".format(str(N)))
    
    for i in range(1,N+2):
        if i>1:
            file.write("P=? [ F (a0=1)")
        else:
            file.write("P=? [ F (a0=0)")

        for j in range(1,N):
            file.write("&(a"+str(j)+"="+str( 1 if i>j+1 else 0 )+")")
        file.write("]\n")
    file.write("R=? [ F b=1] \n")
    file.close()

In [26]:
for N in agents_quantities:
    create_properties(N)

prop_2
prop_3
prop_5
prop_10


<a name="two"></a>

## 2. PARAMETER SYNTHESIS

In this section parameter synthesis is run and the results of it with the data are loaded.

The properties are reachability properties of respective BSCC of the model.

The result are rational function, in this case study polynomes over parameter p,q or p,q1,q2... in multipram case.

In [27]:
os.mkdir("prism_results")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'prism_results'

In [54]:
os.getcwd()

'D:\\Git\\hsb2019\\code'

### Using PRISM

In [60]:
def call_prism(args, seq):
    """  Solves problem of calling prism from another directory.
    
    Parameters
    ----------
    args: string for executing prism
    seq: if true it will take properties by one and append results (neccesary if out of the memory)
    """
    filename = args.split()[0].split(".")[0]+str(".txt")
    filename = os.path.join("prism_results",filename)
    curr_dir = os.getcwd()
    os.chdir(prism_path)
    #print(os.getcwd())
    prism_args = []
    
    try:
        #print(args.split(" "))  

        args=args.split(" ")
        #print(args)
        propfile=args[1]
        #print(propfile)
        for arg in args:
            #print(arg)
            #print(re.compile('\.[a-z]').search(arg))
            if re.compile('\.[a-z]').search(arg) is not None:
                prism_args.append(os.path.join(curr_dir,arg))
                #print(prism_args)
            else:
                prism_args.append(arg)
        #print(prism_args)
        #prism_args.append(" ".join(args.split(" ")[-2:]))
        #print(prism_args)
        
        #print(sys.platform)
        if sys.platform.startswith("win"):
            args=["prism.bat"]
        else:    
            args=["prism"]
        args.extend(prism_args)

        if seq:
            with open(os.path.join(curr_dir,filename), 'a') as f:  
                with open(os.path.join(curr_dir,propfile), 'r') as prop:
                    args.append("-property")                
                    args.append("")
                    prop=prop.readlines()
                    for i in range(1,len(prop)+1):
                        args[-1]= str(i)
                        #print(args)
                        output = subprocess.run(args,stdout=subprocess.PIPE,stderr=subprocess.PIPE).stdout.decode("utf-8") 
                        #print(output)
                        f.write(output)
        else:    
            with open(os.path.join(curr_dir,filename), 'w') as f:
                #print(args)
                output = subprocess.run(args,stdout=subprocess.PIPE,stderr=subprocess.PIPE).stdout.decode("utf-8") 
                #print(output)
                f.write(output)
    finally:
        os.chdir(curr_dir)

Call prism for all files in dir and Saves result to folder `prism_results`. If error with rounding occurs `-noprobchecks` attribute is added. If Java runs out of nemory prism is called with each property separately. 

Example: `prism semisynchronous_parallel_2.txt prop_2.pctl -param p=0:1,q=0:1`

In [117]:
def call_prism_files(file_prefix,multiparam,seq=False,noprobchecks=False):
    if noprobchecks:
        noprobchecks='-noprobchecks '
    else:
        noprobchecks=""
    for N in agents_quantities:
        for file in glob.glob(file_prefix+str(N)+".pm"):
            start_time = time.time()
            print("{} seq={}{}".format(file,seq,noprobchecks))
            if multiparam:
                q=""
                for i in range(1,N):
                    q="{},q{}=0:1".format(q,i)
                    #q=q+",q"+str(i)"=0:1"
            else:
                q=",q=0:1"
            #print("{} prop_{}.pctl {}-param p=0:1{}".format(file,N,noprobchecks,q))
            call_prism("{} prop_{}.pctl {}-param p=0:1{}".format(file,N,noprobchecks,q), seq)
            if not seq: 
                #if 'GC overhead' in tailhead.tail(open('prism_results/{}.txt'.format(file.split('.')[0])),40).read():
                if 'GC overhead' in open('prism_results/{}.txt'.format(file.split('.')[0])).read():
                    seq = True
                    print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")
                    start_time = time.time()
                    call_prism("{} prop_{}.pctl {}-param p=0:1{}".format(file,N,noprobchecks,q), False)
            if not noprobchecks:
                if '-noprobchecks' in open('prism_results/{}.txt'.format(file.split('.')[0])).read():
                    print("An error occured, running with noprobchecks option")
                    noprobchecks='-noprobchecks '
                    print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")
                    start_time = time.time()
                    call_prism("{} prop_{}.pctl {}-param p=0:1{}".format(file,N,noprobchecks,q), False)
            print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")


### Parameter synthesis of synchronous two_param models

In [135]:
call_prism_files("syn*_",False)

synchronous_parallel_2.pm seq=False
  It took Thunder 2.282895803451538 seconds to run
synchronous_parallel_3.pm seq=False
  It took Thunder 2.141301393508911 seconds to run
synchronous_parallel_5.pm seq=False
  It took Thunder 2.3671791553497314 seconds to run
synchronous_parallel_10.pm seq=False
  It took Thunder 2.153200626373291 seconds to run


### Parameter synthesis of semisynchronous two_param models

In [119]:
call_prism_files("semi*_",False)

semisynchronous_parallel_2.pm seq=False
  It took Thunder 2.300849199295044 seconds to run
semisynchronous_parallel_3.pm seq=False
  It took Thunder 2.2050530910491943 seconds to run
semisynchronous_parallel_5.pm seq=False
  It took Thunder 2.1482558250427246 seconds to run
semisynchronous_parallel_10.pm seq=False
  It took Thunder 2.1563894748687744 seconds to run


### Parameter synthesis of asynchronous two_param models

In [120]:
call_prism_files("asyn*_",False)

asynchronous_2.pm seq=False
  It took Thunder 2.3576972484588623 seconds to run
asynchronous_3.pm seq=False
  It took Thunder 2.172189474105835 seconds to run
asynchronous_5.pm seq=False
  It took Thunder 2.423518657684326 seconds to run
asynchronous_10.pm seq=False
  It took Thunder 2.5880792140960693 seconds to run


### Parameter synthesis of synchronous multiparam models

In [136]:
call_prism_files("multiparam_syn*_",True)

multiparam_synchronous_parallel_2.pm seq=False
  It took Thunder 2.142266035079956 seconds to run
multiparam_synchronous_parallel_3.pm seq=False
  It took Thunder 2.1333208084106445 seconds to run
multiparam_synchronous_parallel_5.pm seq=False
  It took Thunder 2.134281873703003 seconds to run
multiparam_synchronous_parallel_10.pm seq=False
  It took Thunder 2.157231092453003 seconds to run


### Parameter synthesis of semisynchronous multiparam models

In [122]:
call_prism_files("multiparam_se*_",True)

multiparam_semisynchronous_parallel_2.pm seq=False
  It took Thunder 2.14426326751709 seconds to run
multiparam_semisynchronous_parallel_3.pm seq=False
  It took Thunder 2.131301164627075 seconds to run
multiparam_semisynchronous_parallel_5.pm seq=False
  It took Thunder 2.1512391567230225 seconds to run
multiparam_semisynchronous_parallel_10.pm seq=False
  It took Thunder 2.322788953781128 seconds to run


### Parameter synthesis of asynchronous multiparam models

In [123]:
call_prism_files("multiparam_a*_",True)

multiparam_asynchronous_2.pm seq=False
  It took Thunder 2.178176164627075 seconds to run
multiparam_asynchronous_3.pm seq=False
  It took Thunder 2.2250471115112305 seconds to run
multiparam_asynchronous_5.pm seq=False
  It took Thunder 2.1782054901123047 seconds to run
multiparam_asynchronous_10.pm seq=False
  It took Thunder 6.811779737472534 seconds to run


All models and properties created, parameter synthesis for all models is done, please follow notebook **Analysis**.